
Targets: Above 98% test_acc with very less number of params  
Results:  
    best test_acc = 99% on 14th epoch  

    total parameters = 5114  
Analysis:  It has Batch normalization, Regularization,  slightly changed model and have GAP
File Link:  

# 1. Import external libraries

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
%matplotlib inline
import matplotlib.pyplot as plt


#### Basically working model
#### Img Aug RandomRotation
#### StepLR Scheduler
#### Train & Test Graphs

# 2. Convolutional Neural Network (model) architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        chan = 8
        adv = 16
        self.conv1 = nn.Conv2d(1, chan, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2 = nn.Conv2d(chan, chan, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(chan, chan, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv4 = nn.Conv2d(chan, chan, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(chan, chan, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv6 = nn.Conv2d(chan, adv, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(num_features=adv, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv7 = nn.Conv2d(adv, 10, 3, padding=1)
        self.gap = nn.AvgPool2d(kernel_size=[10,10], stride=[10,10], padding=0, ceil_mode=False, count_include_pad=False)


        self.drop = nn.Dropout(0.25)

    def forward(self, x):
        x = self.drop(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
        x = self.drop((self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))) # Raja ToDo Try removing dropout here
        x = (F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        x = self.gap(x) # Raja ToDo Try printing shape here
        #print(x.shape)
        x = x.view(-1, 10) # Raja ToDo Try printing shape here
        return F.log_softmax(x)

# 3. Display summary of model

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() #bool
str_gpu_cpu = "cuda" if use_cuda else "cpu" #string
device = torch.device(str_gpu_cpu)
print("device is " + str_gpu_cpu)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

device is cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             584
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
           Dropout-6            [-1, 8, 14, 14]               0
            Conv2d-7            [-1, 8, 14, 14]             584
       BatchNorm2d-8            [-1, 8, 14, 14]              16
            Conv2d-9            [-1, 8, 14, 14]             584
      BatchNorm2d-10            [-1, 8, 14, 14]              16
          Dropout-11            [-1, 8, 14, 14]               0
           Conv2d-12            [-1, 8, 14, 14]             584
      BatchNorm2d-13            [-1, 8, 14, 14]              16
           Conv2d-14    

<ipython-input-2-e79c0a3f2c48>:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


# 4. Preparation of dataset  

In [ ]:

SEED = 1
torch.manual_seed(SEED)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 118028653.56it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38875383.13it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 81855524.35it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 6030556.75it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# 5. Train and Test Functions

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            test_losses.append(test_loss)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\n  Test set: Average loss: {:.4f}, Test Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc.append(100. * correct / len(test_loader.dataset))


# 6. Run the model with a device and an optimizer

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21): #RAJA changed epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

#With params = 3846, on 20th epoch, test accuracy = 98.98%
#With params = 5146, on 11th epoch, test accuracy = 99.20% [reduced afterwards]

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-e79c0a3f2c48>:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
epoch=1 loss=0.40959736704826355 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.46it/s]



  Test set: Average loss: 0.1475, Test Accuracy: 9580/10000 (95.80%)



epoch=2 loss=0.10914165526628494 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.42it/s]



  Test set: Average loss: 0.1418, Test Accuracy: 9577/10000 (95.77%)



epoch=3 loss=0.1423966884613037 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.51it/s]


...


  Test set: Average loss: 0.0293, Test Accuracy: 9903/10000 (99.03%)



epoch=19 loss=0.1599866896867752 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.35it/s]



  Test set: Average loss: 0.0350, Test Accuracy: 9895/10000 (98.95%)



epoch=20 loss=0.15257926285266876 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.72it/s]



  Test set: Average loss: 0.0383, Test Accuracy: 9884/10000 (98.84%)



Raja ToDo :
Try below ::
1. BatchNormalization
2. Dropout
3. LR scheduler
4. GAP